In [15]:
import random

import pandas as pd

In [33]:
# import data
path_data_train = "/Users/jerome/Documents/level_up/ML/data/sentiment_train.csv"
train = pd.DataFrame.from_csv(path_data_train, encoding = "ISO-8859-1", index_col = None)
path_data_test = "/Users/jerome/Documents/level_up/ML/data/sentiment_test.csv"
test = pd.DataFrame.from_csv(path_data_test, encoding = "ISO-8859-1", index_col = None)
train.iloc[1]["tweet"]

'@SouthwestAir cool the plane you said is ours that just left the terminal for a diff city? She a cold betch for "flying the friendly skies"'

In [36]:
texts_train = train['tweet'].fillna("").values
labels_train = train['sentiment'].values

texts_test = train['tweet'].fillna("").values
labels_test = train['sentiment'].values

from nltk.stem import SnowballStemmer
from nltk import RegexpTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
import re

stemmer = SnowballStemmer("english")

def isNumeric(word):
    try:
        tmp = float(word)
        return True
    except Exception as e:
        return False

def preprocessing(word):
    if isNumeric(word):
        return "_numeric_"
    elif re.search("^@", word):
        return "_username_"
    else:
        return stemmer.stem(word)

tokenizer = RegexpTokenizer('@?\w+', flags = re.UNICODE, gaps = False)

new_texts_train = []
for tweet in texts_train:
    tweet = " ".join([preprocessing(word) for word in tokenizer.tokenize(tweet)])
    new_texts_train.append(tweet)
print(new_texts_train[:5])

new_texts_test = []
for tweet in texts_train:
    tweet = " ".join([preprocessing(word) for word in tokenizer.tokenize(tweet)])
    new_texts_test.append(tweet)   

tfidf = TfidfVectorizer(min_df=5)

X_train = tfidf.fit_transform(new_texts_train)
print(X_train.shape)
X_test = tfidf.transform(new_texts_test)
print(X_test.shape)
    

['_username_ flight _numeric_ in d fw now delay _numeric_ hour wait sit in plane for baggag to be load whi', '_username_ cool the plane you said is our that just left the termin for a diff citi she a cold betch for fli the friend sky', '_username_ well then that is a horribl flaw in your system _numeric_ hour in the past day i ve wast i ve got a job amp famili this is crazi', '_username_ thank i think we ve got it figur out', '_username_ don t want to clog your alreadi crowd line with a special meal request can someon dm me to sort out vegetarianproblem']
(22633, 3632)
(22633, 3632)


In [42]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

logistic = LogisticRegression()
kfold = KFold(n_splits=3, shuffle=True)

parameters = {'C': [0.01, 0.1, 1, 10]}
scoring = "accuracy"

cross = GridSearchCV(logistic, parameters, 
                     scoring=scoring,
                     cv=kfold, 
                     verbose=2
                    )
cross.fit(X_train, labels_train)
print(cross.best_params_)
print(cross.best_score_)

y_pred = cross.predict(X_test)

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

print(classification_report(labels_test, y_pred))
print(confusion_matrix(labels_test, y_pred))
print(accuracy_score(labels_test, y_pred))




Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] C=0.01 ..........................................................
[CV] ........................................... C=0.01, total=   0.2s
[CV] C=0.01 ..........................................................
[CV] ........................................... C=0.01, total=   0.1s
[CV] C=0.01 ..........................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


[CV] ........................................... C=0.01, total=   0.1s
[CV] C=0.1 ...........................................................
[CV] ............................................ C=0.1, total=   0.2s
[CV] C=0.1 ...........................................................
[CV] ............................................ C=0.1, total=   0.2s
[CV] C=0.1 ...........................................................
[CV] ............................................ C=0.1, total=   0.2s
[CV] C=1 .............................................................
[CV] .............................................. C=1, total=   0.3s
[CV] C=1 .............................................................
[CV] .............................................. C=1, total=   0.3s
[CV] C=1 .............................................................
[CV] .............................................. C=1, total=   0.3s
[CV] C=10 ............................................................
[CV] .

[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    3.6s finished


{'C': 1}
0.749348296735
             precision    recall  f1-score   support

   negative       0.84      0.90      0.87      9324
    neutral       0.75      0.82      0.78      8587
   positive       0.80      0.56      0.66      4722

avg / total       0.80      0.80      0.79     22633

[[8363  774  187]
 [1097 7028  462]
 [ 521 1558 2643]]
0.796801131092


In [44]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

count = CountVectorizer(
            min_df=5)

X_train_count = count.fit_transform(new_texts_train)
X_test_count = count.transform(new_texts_test)

multinomial = MultinomialNB()

parameters = {'alpha': [0, 0.2, 0.4, 0.6, 0.8, 1]}

cross = GridSearchCV(multinomial, parameters, 
                     scoring=scoring,
                     cv=kfold, 
                     verbose=2
                    )

cross.fit(X_train_count, labels_train)
print(cross.best_params_)
print(cross.best_score_)

y_pred = cross.predict(X_test_count)

print(classification_report(labels_test, y_pred))
print(confusion_matrix(labels_test, y_pred))
print(accuracy_score(labels_test, y_pred))


Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV] alpha=0 .........................................................
[CV] .......................................... alpha=0, total=   0.1s
[CV] alpha=0 .........................................................
[CV] .......................................... alpha=0, total=   0.1s
[CV] alpha=0 .........................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


[CV] .......................................... alpha=0, total=   0.1s
[CV] alpha=0.2 .......................................................
[CV] ........................................ alpha=0.2, total=   0.1s
[CV] alpha=0.2 .......................................................
[CV] ........................................ alpha=0.2, total=   0.1s
[CV] alpha=0.2 .......................................................
[CV] ........................................ alpha=0.2, total=   0.1s
[CV] alpha=0.4 .......................................................
[CV] ........................................ alpha=0.4, total=   0.1s
[CV] alpha=0.4 .......................................................
[CV] ........................................ alpha=0.4, total=   0.1s
[CV] alpha=0.4 .......................................................
[CV] ........................................ alpha=0.4, total=   0.1s
[CV] alpha=0.6 .......................................................
[CV] .

[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed:    1.3s finished


[[8420  634  270]
 [1958 5552 1077]
 [ 850 1000 2872]]
0.744223037158


In [48]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', LogisticRegression()),
])

parameters = {
    'vect__min_df': (1, 5, 10),
    'vect__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
    'clf__C': (0.1, 1, 10)
}

grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=2, cv=kfold)
grid_search.fit(new_texts_train,labels_train)

print(grid_search.best_params_)
print(grid_search.best_score_)

y_pred = grid_search.predict(new_texts_test)

print(classification_report(labels_test, y_pred))
print(confusion_matrix(labels_test, y_pred))
print(accuracy_score(labels_test, y_pred))



Fitting 3 folds for each of 18 candidates, totalling 54 fits
[CV] clf__C=0.1, vect__min_df=1, vect__ngram_range=(1, 1) ............
[CV] clf__C=0.1, vect__min_df=1, vect__ngram_range=(1, 1) ............
[CV] clf__C=0.1, vect__min_df=1, vect__ngram_range=(1, 1) ............
[CV] clf__C=0.1, vect__min_df=1, vect__ngram_range=(1, 2) ............
[CV]  clf__C=0.1, vect__min_df=1, vect__ngram_range=(1, 1), total=   1.6s
[CV]  clf__C=0.1, vect__min_df=1, vect__ngram_range=(1, 1), total=   1.7s
[CV] clf__C=0.1, vect__min_df=1, vect__ngram_range=(1, 2) ............
[CV]  clf__C=0.1, vect__min_df=1, vect__ngram_range=(1, 1), total=   1.7s
[CV] clf__C=0.1, vect__min_df=1, vect__ngram_range=(1, 2) ............
[CV] clf__C=0.1, vect__min_df=5, vect__ngram_range=(1, 1) ............
[CV]  clf__C=0.1, vect__min_df=5, vect__ngram_range=(1, 1), total=   1.4s
[CV] clf__C=0.1, vect__min_df=5, vect__ngram_range=(1, 1) ............
[CV]  clf__C=0.1, vect__min_df=1, vect__ngram_range=(1, 2), total=   4.1s
[

[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   36.6s


[CV]  clf__C=10, vect__min_df=1, vect__ngram_range=(1, 1), total=   2.5s
[CV] clf__C=10, vect__min_df=1, vect__ngram_range=(1, 1) .............
[CV]  clf__C=1, vect__min_df=10, vect__ngram_range=(1, 2), total=   4.4s
[CV] clf__C=10, vect__min_df=1, vect__ngram_range=(1, 1) .............
[CV]  clf__C=1, vect__min_df=10, vect__ngram_range=(1, 2), total=   4.4s
[CV] clf__C=10, vect__min_df=1, vect__ngram_range=(1, 2) .............
[CV]  clf__C=1, vect__min_df=10, vect__ngram_range=(1, 2), total=   4.5s
[CV] clf__C=10, vect__min_df=1, vect__ngram_range=(1, 2) .............
[CV]  clf__C=10, vect__min_df=1, vect__ngram_range=(1, 1), total=   3.3s
[CV] clf__C=10, vect__min_df=1, vect__ngram_range=(1, 2) .............
[CV]  clf__C=10, vect__min_df=1, vect__ngram_range=(1, 1), total=   2.9s
[CV] clf__C=10, vect__min_df=5, vect__ngram_range=(1, 1) .............
[CV]  clf__C=10, vect__min_df=5, vect__ngram_range=(1, 1), total=   2.6s
[CV] clf__C=10, vect__min_df=5, vect__ngram_range=(1, 1) ......

[Parallel(n_jobs=-1)]: Done  54 out of  54 | elapsed:  1.1min finished


{'clf__C': 10, 'vect__min_df': 1, 'vect__ngram_range': (1, 2)}
0.763266027482
             precision    recall  f1-score   support

   negative       0.99      0.99      0.99      9324
    neutral       0.96      0.99      0.97      8587
   positive       0.98      0.94      0.96      4722

avg / total       0.98      0.98      0.98     22633

[[9189  126    9]
 [  48 8466   73]
 [   9  264 4449]]
0.976627048999
